In [2]:
import Agent_Library
from   Agent_Library import np, plt, Camera, tf, nx

import random
import pandas as pd
from openpyxl import Workbook

import importlib
# import psutil
# import cProfile, pstats
# import snakeviz.cli as cli
# %load_ext snakeviz

In [4]:
def Initializer(N, L, seed=42, requesting=False):
    importlib.reload(Agent_Library)
    np.random.seed(seed)
    random.seed(seed)
    # -----------------------------------------------------------------------------------
    camera = Camera(plt.figure(figsize=(15,15)))

    # Build the environment and the agents ----------------------------------------------
    Agents = []
    for _ in range(N): 
        Agents.append(Agent_Library.Distributed_Agent(N, L, flipping=True, requesting=requesting, moving=False, training=False))
    
    environment = Agent_Library.Plot_Environment(L, Agents)

    
    return camera, environment, Agents

In [ ]:
def ExportResults(step, environment, Agents, AveragedExcel, ParticleExcel):    
    environment.Environmental_Changes(Agents)
    G = nx.from_numpy_array(environment.A)
    Giant = sorted(nx.connected_components(G), key=len, reverse=True)
     
    # Show Averaged Results ---------------------------------------------------------------------
    # step	Hamilton	Giant	Edges	Energy	Tau	R_avg ---------------------------------------
    hamilton, edge, energy, average_r, giant, tau = environment.Calculate_Result(Agents, step)
    AveragedExcel.append([step, hamilton, giant, edge, energy, tau, average_r])


    # Show Particle Results ---------------------------------------------------------------------
    # step	index	X	Y	Component	r	k	rho	Energy	connected_to ------------------------
    for i in range(len(Agents)):
        particle_hamilton = Agents[i].Hamiltonian()
        particle_k, particle_r, particle_rho = Agents[i].MyState()

        for j in range(len(Giant)):
            if i in Giant[j]: component = j

        co = []
        for j in range(len(Agents)):
            if environment.A[i][j] == 1: co.append(j) 
        ParticleExcel.append([step, i, Agents[i].x, Agents[i].y, component, 
                                particle_r, particle_k, particle_rho, particle_hamilton, str(co)])

# MAIN

In [4]:
def base_model(seed, ParticleExcel, AveragedExcel):
    N = 100                      # Number of agents
    L = 14                       # The length of the simulation box
    _, environment, Agents = Initializer(N, L, seed)

    ParticleExcel_ = ParticleExcel.create_sheet(title=f'Base')
    AveragedExcel_ = AveragedExcel.create_sheet(title=f'Base')
    ParticleExcel_.append(["step", "index", "X", "Y", "Component", "r", "k", "rho", "Hamilton", "connected to"])
    AveragedExcel_.append(["step", "Hamilton", "Giant", "Edges", "Energy", "Tau", "R_avg"])

    for step in range(1001):
        for i in range(len(Agents)):
            Agents[i].N = len(Agents)
            Agents[i].OtherAgents = (Agents[:i] + Agents[i+1:])
            if Agents[i].k < 5: action = np.random.randint(2)
            else: action = 0
        
            _, _ = Agents[i].step(action)


        ExportResults(step, environment, Agents, AveragedExcel, ParticleExcel)
            
            
        # -------------------------------------------------------------------------------------------
        print("\rSeed: {}, Step: {}".format(seed, step), end="")


In [5]:
def AI_model(seed, ParticleExcel, AveragedExcel, requesting):
    N = 100                      # Number of agents
    L = 14                       # The length of the simulation box
    _, environment, Agents = Initializer(N, L, seed, requesting)


    if requesting:  sheet_name = "Send Request"
    else:           sheet_name = "Just Model"

    ParticleExcel_ = ParticleExcel.create_sheet(title=sheet_name)
    AveragedExcel_ = AveragedExcel.create_sheet(title=sheet_name)
    ParticleExcel_.append(["step", "index", "X", "Y", "Component", "r", "k", "rho", "Hamilton", "connected to"])
    AveragedExcel_.append(["step", "Hamilton", "Giant", "Edges", "Energy", "Tau", "R_avg"])


    for step in range(1001):
        for i in range(len(Agents)):
            Agents[i].Prediction( (Agents[:i] + Agents[i+1:]) )

        
        ExportResults(step, environment, Agents, AveragedExcel, ParticleExcel)
            

        # -------------------------------------------------------------------------------------------
        print("\rSeed: {}, Step: {}".format(seed, step), end="")


In [ ]:
for seed in range(30, 35):
    
    # ParticleExcel = pd.ExcelWriter(f"./All result/csv/Particle data {seed}.xlsx", engine='openpyxl')
    # AveragedExcel = pd.ExcelWriter(f"./All result/csv/Averaged data {seed}.xlsx",  engine='openpyxl')
    ParticleExcel = Workbook(); del ParticleExcel['Sheet']
    AveragedExcel = Workbook(); del AveragedExcel['Sheet']


    df_base   = base_model(seed, ParticleExcel, AveragedExcel)       ;      print("  finish Base...")
    df_AI     =   AI_model(seed, ParticleExcel, AveragedExcel, False);      print("  finish False...") 
    df_AI_Req =   AI_model(seed, ParticleExcel, AveragedExcel, True );      print("  finish True!")
    

    ParticleExcel.save(f"./datas/data {seed}/{seed} Particle Data - Movement & Train 0.001.xlsx")
    AveragedExcel.save(f"./datas/data {seed}/{seed} Averaged Data - Movement & Train 0.001.xlsx")

    # ParticleExcel.close()
    # AveragedExcel.close()
    

# Learning Model

In [ ]:
N = 100                      # Number of agents
L = 15                       # The length of the simulation box
camera, environment, Agents = Initializer(N, L)

model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(32, activation="elu", input_shape=[3]),
            tf.keras.layers.Dense(32, activation="elu"),
            tf.keras.layers.Dense(2,  activation=Agents[0].custom_activation)
        ])

for i in range(N): 
    Agents[i].model = model
    Agents[i].OtherAgents = (Agents[:i] + Agents[i+1:])


ParticleExcel = Workbook()
AveragedExcel = Workbook()
ParticleExcel.append(["step", "index", "X", "Y", "Component", "r", "k", "rho", "Hamilton", "connected to"])
AveragedExcel.append(["step", "Hamilton", "Giant", "Edges", "Energy", "Tau", "R_avg"])

best_H, best_G, best_H2G2 = 0, 0, 0



# -----------------------------------------------------------------------------------------------
for step in range(1001):
    
    epsilon = max(1 - step/500, 0.0)             # first is more random and than use greedy
    for _ in range(10):
    # Like play_one_step function ---------------------------------------------------------------
        for i in range(len(Agents)):

            if np.random.rand() < epsilon:
                state  = [s+1 for s in Agents[i].MyState()]
                action = np.random.randint(2)
                STEP   = Agents[i].step((action-0.5)*2)
                reward = STEP[1]
                next_state = [ns+1 for ns in STEP[0]]

                Agents[i].replay_memory.append((state, action, reward, next_state))

            else: 
                Agents[i].Prediction( (Agents[:i] + Agents[i+1:]) )


    # Result Section ----------------------------------------------------------------------------       
    ExportResults(step, environment, Agents, AveragedExcel, ParticleExcel)
    hamilton, edge, energy, average_r, giant, tau = environment.Calculate_Result(Agents, step)

    # Find Best Weights -------------------------------------------------------------------------     
    if step > 500:
        if hamilton <= best_H:                                  # find the minimum of Hamiltonian
            H_best_weight    = Agents[0].model.get_weights()    # saving model weights for the best Hamiltonian
            best_H = hamilton

        if giant >= best_G:
            G_best_weight    = Agents[0].model.get_weights()    # saving model weights for the best Giant Component
            best_G = giant

        if np.sqrt(hamilton**2 + giant**2) >= best_H2G2:
            H2G2_best_weight = Agents[0].model.get_weights()
            best_H2G2 = np.sqrt(hamilton**2 + giant**2)



    # Train Section -----------------------------------------------------------------------------
    if step > 50:
        for i in range(N):
            Agents[i].Train(batch_size=20, learning_rate=0.01)
            if i != N: Agents[i+1].model = Agents[i].model

        for i in range(N-1): Agents[i].model = Agents[N].model


    print("\rStep: {}, Epsilon: {:.3f}, Min(Hamilton): {:.3f}, H: {:.3f}".format(step, epsilon, best_H, hamilton), end="")

last_weight = Agents[0].model.get_weights()


# Save ------------------------------------------------------------------------------------------
ParticleExcel.save(f"./All result/models with alpha4 = -1000/ Particle Data.xlsx")
AveragedExcel.save(f"./All result/models with alpha4 = -1000/ Averaged Data.xlsx")


model_H_best_weight    = model;    model_H_best_weight.set_weights(H_best_weight)
model_G_best_weight    = model;    model_G_best_weight.set_weights(G_best_weight)
model_H2G2_best_weight = model;    model_H2G2_best_weight.set_weights(H2G2_best_weight)
model_last_weight      = model;    model_last_weight.set_weights(last_weight)

model_H_best_weight   .save('./All result/models with alpha4 = -1000/model_H_best_weight.keras')
model_G_best_weight   .save('./All result/models with alpha4 = -1000/model_G_best_weight.keras')
model_H2G2_best_weight.save('./All result/models with alpha4 = -1000/model_H2G2_best_weight.keras')
model_last_weight     .save('./All result/models with alpha4 = -1000/model_last_weight.keras')

# Make Animation

In [2]:
import ast
import gc
import warnings
warnings.filterwarnings("ignore")

N = 100
L = 14

sheets = ['Just Model', 'Send Request']
for seed in range(30,31):
    for sheet in sheets:
        
        result = pd.read_excel(f"./datas/data {seed}/{seed} Particle Data - Movement & Train.xlsx", sheet_name=sheet)

        fig = plt.figure(figsize=(14,14))
        fig.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95, wspace=None, hspace=None)
        camera = Camera(fig)

        for step in range(1001):
            df = result["connected to"][step*N:(step+1)*N].apply(lambda cell: ast.literal_eval(cell))        # string to list

            A = np.zeros((N, N))
            for i in range(N):
                for l in df[step*N + i]:
                    A[i][l] = 1

            # Ploting -------------------------------------------------------------------------------
            if step%5==0:
                G = nx.from_numpy_array(A)

                options = { 'node_size': 60, 'width': 1.5, 'node_color': (0.5,0.0,0.8,1) }
                for i in range(N): G.add_node(i, pos=(result["X"][step*N+i], result["Y"][step*N+i]))
                pos = nx.get_node_attributes(G, 'pos')
                poss = dict( (i, ( result["X"][step*N+i]+0.2, result["Y"][step*N+i]+0.1 )) for i in range(N))
                nx.draw_networkx_labels(G, poss, font_size=10.5, font_weight='bold', font_color='k')

                nx.draw_networkx(G, pos, with_labels=False, **options)

                plt.text(L-0.1*L, L+0.5, f'Episode {step}', fontname='Comic Sans MS', fontsize=13)
                plt.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
                plt.xlim(-0.3, L+0.3); plt.ylim(-0.3, L+0.3)
                plt.grid(alpha=0.3)
                
                camera.snap()

            print("\rSeed: {}, Sheep: {}".format(seed, sheet), end="")

        anim = camera.animate(interval= 120, repeat=True, repeat_delay= 500, blit=True)
        anim.save(f'./datas/data {seed}/{seed} Animation - Movement & Train -- {sheet}.gif')
        print()

        del result
        plt.close(fig)
        gc.collect()

Seed: 30, Sheep: Just Model
Seed: 30, Sheep: Send Request


# Test

In [ ]:
# with cProfile.Profile() as profile: main()
# stats = pstats.Stats(profile).sort_stats('cumtime')
# stats.dump_stats("./All result/cProFile/cProFile.prof")

# stats.print_stats()
# cli.main(["./All result/cProFile/cProFile.prof"])

In [61]:
N = 100                      # Number of agents
L = 14                       # The length of the simulation box
# camera, environment, Agents, Hamilton, Giant, Edges, Energy, R_avg, Tau = Initializer(N, L, 30)
Agents[0].N = len(Agents)
Agents[0].OtherAgents = (Agents[:0] + Agents[1:])


next_state = Agents[0].step(0)[0]
# next_state = [ns+1 for ns in next_state]
next_state



[2, 1.0, 5]